In [1]:
import logging

import probtorch
import torch

import combinators
import smc
import hmm
import utils

logging.basicConfig(format='%(asctime)s %(message)s', datefmt='%m/%d/%Y %H:%M:%S',
                    level=logging.INFO)

In [2]:
generative = combinators.ParticleTrace(1)

In [3]:
hmm_params = utils.vardict({
    'mu': {
        'loc': torch.arange(5, dtype=torch.float).unsqueeze(0) * 2,
        'scale': torch.ones(1, 5) * 0.25,
    },
    'sigma': {
        'loc': torch.ones(1, 5),
        'scale': torch.ones(1, 5) * 0.25,
    }
})
for k in range(6):
    hmm_params['Pi_%d' % k] = {'concentration': torch.ones(1, 5)}

In [4]:
init_hmm = combinators.Model(f=hmm.init_hmm, theta=hmm_params)
init_hmm.condition(generative)

In [5]:
z0, mu, sigma, pi, pi0 = init_hmm()

In [6]:
hmm_step = combinators.Model(f=hmm.hmm_step)
hmm_run = combinators.Model.sequence(hmm_step, 50, z0, mu, sigma, pi, pi0)

In [7]:
hmm_run.condition(generative)

In [8]:
z_last, mu, sigma, pi, pi0 = hmm_run()

In [9]:
generative = generative.squeeze()

In [10]:
num_particles = 100

In [11]:
smc_hmm_params = utils.vardict({
    'mu': {
        'loc': torch.arange(5, dtype=torch.float) * 2,
        'scale': torch.ones(5),
    },
    'sigma': {
        'loc': torch.ones(5),
        'scale': torch.ones(5),
    }
})
for k in range(6):
    smc_hmm_params['Pi_%d' % k] = {'concentration': torch.ones(5)}

In [12]:
smc_hmm = smc.smc(hmm.hmm_step, 50)

In [13]:
init_hmm = combinators.Model(hmm.init_hmm, phi=smc_hmm_params)

In [14]:
samples, elbos, inference = smc.particle_mh(num_particles, init_hmm, smc_hmm, 1000, generative, use_cuda=False)

In [15]:
for t in range(50):
    t += 1
    key = 'Z_%d' % t
    accuracy = (inference[key].value == generative[key].value).sum().to(dtype=torch.float) / inference.num_particles
    print('SMC accuracy at time %d: %f' % (t, accuracy))

SMC accuracy at time 1: 0.000000
SMC accuracy at time 2: 0.000000
SMC accuracy at time 3: 0.030000
SMC accuracy at time 4: 1.000000
SMC accuracy at time 5: 0.970000
SMC accuracy at time 6: 0.740000
SMC accuracy at time 7: 0.030000
SMC accuracy at time 8: 1.000000
SMC accuracy at time 9: 0.030000
SMC accuracy at time 10: 0.000000
SMC accuracy at time 11: 0.970000
SMC accuracy at time 12: 0.000000
SMC accuracy at time 13: 1.000000
SMC accuracy at time 14: 0.000000
SMC accuracy at time 15: 0.540000
SMC accuracy at time 16: 0.620000
SMC accuracy at time 17: 0.590000
SMC accuracy at time 18: 0.510000
SMC accuracy at time 19: 0.560000
SMC accuracy at time 20: 0.330000
SMC accuracy at time 21: 0.000000
SMC accuracy at time 22: 0.560000
SMC accuracy at time 23: 0.030000
SMC accuracy at time 24: 0.540000
SMC accuracy at time 25: 0.030000
SMC accuracy at time 26: 0.330000
SMC accuracy at time 27: 0.790000
SMC accuracy at time 28: 0.560000
SMC accuracy at time 29: 0.410000
SMC accuracy at time 30

In [16]:
elbos

tensor([-249.8494, -245.7230, -245.7230, -245.7230, -245.7230, -245.7230,
        -245.1366, -245.1366, -245.1366, -245.1366, -245.1366, -245.1366,
        -242.2717, -235.8921, -235.8921, -235.8921, -235.8921, -235.8921,
        -235.8921, -235.8921, -235.8921, -235.8921, -235.8921, -235.8921,
        -235.8921, -235.8921, -235.8921, -235.8921, -235.8921, -235.8921,
        -235.8921, -235.8921, -235.8921, -235.8921, -237.0874, -236.1996,
        -239.4027, -239.4027, -239.4027, -239.4027, -239.4027, -239.4027,
        -239.4027, -239.4027, -239.4027, -239.4027, -239.4027, -239.4027,
        -239.4027, -235.4757, -235.4757, -235.4757, -235.4757, -235.4757,
        -235.4757, -235.4757, -235.4757, -235.4757, -235.4757, -235.4757,
        -235.4757, -235.4757, -235.4757, -235.4757, -235.4757, -235.4757,
        -235.4757, -235.4757, -235.4757, -235.4757, -235.4757, -235.4757,
        -235.4757, -234.3843, -234.3843, -234.3843, -234.3843, -234.3843,
        -234.3843, -234.3843, -234.384